# 🗺️ Spatial Analysis

Geographic patterns and spatial visualization of weather data.

In [1]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
import warnings
warnings.filterwarnings('ignore')
print("✅ Libraries loaded!")

✅ Libraries loaded!


In [2]:
# Load CLEANED data
df = pd.read_csv("../data/weather_cleaned.csv", parse_dates=['last_updated'])
print(f"📊 Dataset: {len(df):,} records, {df['country'].nunique()} countries")

📊 Dataset: 114,203 records, 204 countries


In [3]:
# Global temperature map
location_avg = df.groupby(['latitude', 'longitude']).agg({'temperature_celsius': 'mean', 'location_name': 'first', 'country': 'first'}).reset_index()
fig = px.scatter_geo(location_avg.sample(min(5000, len(location_avg))), lat='latitude', lon='longitude', color='temperature_celsius', hover_name='location_name', title='🌍 Global Temperature Distribution', color_continuous_scale='RdYlBu_r')
fig.update_layout(template='plotly_dark', height=600)
fig.show()

In [4]:
# Temperature by latitude
df['lat_bin'] = pd.cut(df['latitude'], bins=18, labels=[f"{i*10-90} to {(i+1)*10-90}" for i in range(18)])
lat_temp = df.groupby('lat_bin')['temperature_celsius'].mean().reset_index()
fig = px.bar(lat_temp, x='lat_bin', y='temperature_celsius', title='🌡️ Temperature by Latitude Band')
fig.update_layout(template='plotly_dark', height=400, xaxis_tickangle=-45)
fig.show()

In [5]:
# Country statistics
country_stats = df.groupby('country').agg({'temperature_celsius': ['mean', 'std', 'count'], 'humidity': 'mean'}).round(2)
country_stats.columns = ['Temp Mean', 'Temp Std', 'Records', 'Humidity']
country_stats = country_stats[country_stats['Records'] >= 100].sort_values('Temp Mean', ascending=False)
print("🌍 Top 10 Hottest Countries:")
country_stats.head(10)

🌍 Top 10 Hottest Countries:


,Temp Mean,Temp Std,Records,Humidity
country,,,,
Qatar,33.82,7.17,586,38.56
United Arab Emirates,33.68,6.46,587,42.11
Kuwait,33.22,9.99,587,24.08
Saudi Arabia,32.85,9.48,586,15.66
Djibouti,32.38,4.67,585,53.41
Oman,32.19,4.85,587,52.05
Bahrain,31.45,7.08,587,47.39
Cambodia,31.20,2.90,586,66.68
Iraq,31.19,10.92,587,26.40


In [6]:
# Choropleth map
fig = px.choropleth(country_stats.reset_index(), locations='country', locationmode='country names', color='Temp Mean', title='🌍 Average Temperature by Country', color_continuous_scale='RdYlBu_r')
fig.update_layout(template='plotly_dark', height=500)
fig.show()